In [97]:
import os
import email
import csv
from datetime import datetime

In [120]:
path = os.path.join('.', 'enron_mail_20150507', 'maildir')
csv_file_path = os.path.join('.', 'mails.csv')
date_format = '%a, %d %b %Y %H:%M:%S %z'

def list_all_sent_email(folder_path):
    path = folder_path
    sent_mails = []
    if os.path.exists(path):
        for name in os.listdir(path):
            person_folder = os.path.join(path, name)
            sent_folder = os.path.join(person_folder, 'sent')
            if os.path.isdir(person_folder) and os.path.exists(sent_folder) and os.path.isdir(sent_folder):
                for name in os.listdir(sent_folder):
                    file_path = os.path.join(sent_folder, name)
                    if os.path.isfile(file_path):
                        sent_mails.append(file_path)
    return sent_mails
     
def extract_fields(mail_content):
    data = []
    msg = email.message_from_string(mail_content)
    data.append(msg['Subject'])
    data.append(datetime.strptime(msg['Date'][:-6], date_format))
    data.append(msg.get_payload(decode=True).decode().replace('\t','').replace('\n', ''))

    return data
    
def read_mail(mail_path):
    data = ''
    with open(mail_path, 'r') as f:
        data = f.read()
    return data

def extract_emails(mail_list):
    data_list = []
    for mail_path in mail_list:
        data = read_mail(mail_path)
        data_list.append(extract_fields(data))
    return data_list

def export_to_csv(csv_file_path, mail_list):
    headers = ['Subject', 'Date', 'Body']
    data = extract_emails(mail_list)
    with open(csv_file_path, 'w', newline='') as f:
        csv_writer = csv.writer(f, delimiter='\t')
        csv_writer.writerow(headers)
        for row in data:
            csv_writer.writerow(row)

In [121]:
#mail_list = list_all_sent_email(path)
export_to_csv(csv_file_path, mail_list[:4])

In [122]:
import pandas as pd

In [123]:
df = pd.read_csv('mails.csv', delimiter='\t')
df

,Subject,Date,Body
0,NaN,2000-12-13 07:04:00-08:00,Attached are two files that illustrate the fo...
1,Headcount,2000-12-05 07:31:00-08:00,Financial (6) West Desk (14)Mid Market (16)
2,utilities roll,2000-09-06 08:54:00-07:00,---------------------- Forwarded by Phillip K ...
3,TIME SENSITIVE: Executive Impact & Influence P...,2000-09-06 07:02:00-07:00,---------------------- Forwarded by Phillip K ...


In [105]:
from datetime import datetime

# Date string
date_string = 'Wed, 13 Dec 2000 07:04:00 -0800 (PST)'

# Define the format of the date string
date_format = '%a, %d %b %Y %H:%M:%S %z'

# Convert the date string to a datetime object
datetime_obj = datetime.strptime(date_string[:-6], date_format)

# Print the datetime object
print(datetime_obj)


2000-12-13 07:04:00-08:00
